In [6]:
import utils
from importlib import reload 
reload(utils)
from utils import * 
import math
import PyQt6

a, b, c = symbols("a b c")
x, y = symbols("x y")
s, t = symbols("s t")

invariant1 = x * y 
invariant_liq1 = [(x, 10), (y,10)]
invariant2 = s * t 
invariant_liq2 = [(s, 15), (t,25)]
invariant3 = x**2 * y 
invariant_liq3 = [(x, 42), (y,35)]
# invariant3 = x * y 
# invariant_liq3 = [(x, 21), (y,35)]
invariant_liq_display1 = np.array([2, 20])
invariant_liq_display2 = np.array([5, 20])
invariant_liq_display3 = np.array([15, 45])
invariant_liq_displayc = np.array([5, 20])
invariant_liq_displays = np.array([0.5, 7.0]) # Parametric, in terms of price 

delta_f1 = invariant_to_delta(invariant1, invariant_liq1, a)
risk_av1 = 2
U_f1 = 2*b + 5*c

omega_1 = [8,2]
RES = 100


Most graphs in the paper can be reproduced by running this notebook.

In [7]:

def optim_swap_figure():
    fig = plt.figure(facecolor='white',figsize=(9,6))
    budget_curve_x, budget_curve_y = budget_curve(delta_f1, omega_1, a, resolution=RES)
    tangency_point, max_utils = opt_swap_2(delta_f1, U_f1, omega_1, [a,b,c])
    init_utils = U_f1.subs([(b, omega_1[0]), (c, omega_1[1])])
    budget_curve_interval_x = (min(budget_curve_x), max(budget_curve_x))
    indiff_curves = [indifference_curve(U_f1, max_utils, budget_curve_interval_x, [b, c], resolution=RES), \
        indifference_curve(U_f1, init_utils, budget_curve_interval_x, [b, c], resolution=RES)]

    tngt = plt.scatter(tangency_point[0], tangency_point[1], color='k', marker='o', s=70)
    init = plt.scatter(omega_1[0], omega_1[1], color='k', marker='s', s=70)
    bdgt, = plt.plot(budget_curve_x, budget_curve_y, 'k')
    for (curve_x, curve_y) in indiff_curves:
        indf, = plt.plot(curve_x, curve_y, 'k--')

    plt.xlabel(r"$\theta_1 + \Delta_1$")
    plt.ylabel(r"$\theta_2 + \Delta_2$")
    plt.legend([bdgt, indf, init, tngt], ['Budget Line', 'Indifference Curves', 'Initial Endowment', 'Max Utility'])
    plt.show()
    fig.savefig('outputs/optim_swap.png')


def invar_curve1_figure():
    fig = plt.figure(facecolor='white',figsize=(9,6))
    x_range, y_range = invariant_curve(invariant1, invariant_liq_display1, invariant_liq1, [x,y])
    invariant = plt.plot(x_range, y_range, 'k')
    init = plt.scatter(invariant_liq1[0][1], invariant_liq1[1][1], marker='s', s=70, color='k')
    plt.xlabel(r"$r_{12}$")
    plt.ylabel(r"$r_{21}$")
    plt.text(x=7.5,y=20, s=r"$x \cdot y = k$", size=25)
    plt.legend([invariant, init], ['Invariance Curve', 'Initial Liquidity'])
    plt.show()
    fig.savefig('outputs/invar_curve1.png')


def chained_curve_figures():
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, facecolor='white',figsize=(27,6))
    x_range, y_range = invariant_curve(invariant1, invariant_liq_display1, invariant_liq1, [x,y])
    invariant = ax1.plot(x_range, y_range, 'k')
    init = ax1.scatter(invariant_liq1[0][1], invariant_liq1[1][1], marker='s', s=70, color='k')
    ax1.set_xlabel(r"$r_{13}$")
    ax1.set_ylabel(r"$r_{31}$")

    x_range, y_range = invariant_curve(invariant2, invariant_liq_display2, invariant_liq2, [s,t])
    invariant = ax2.plot(x_range, y_range, 'k')
    init = ax2.scatter(invariant_liq2[0][1], invariant_liq2[1][1], marker='s', s=70, color='k')
    ax2.set_xlabel(r"$r_{32}$")
    ax2.set_ylabel(r"$r_{23}$")

    invariantc, invariant_liqc = chain_pools(invariant1, invariant2, invariant_liq1, invariant_liq2)
    chained_liq_xr, chained_liq_yr = invariant_curve(invariantc, invariant_liq_displayc, invariant_liqc, [v[0] for v in invariant_liqc])
    invariant = ax3.plot(chained_liq_xr, chained_liq_yr, 'k')
    init = ax3.scatter(invariant_liqc[0][1], invariant_liqc[1][1], marker='s', s=70, color='k')
    ax3.set_xlabel(r"$r_{12}$")
    ax3.set_ylabel(r"$r_{21}$")

    plt.subplots_adjust(wspace=0.6)

    plt.show()
    fig.savefig('outputs/chained_curve.png')

    # plt.subplots_adjust(hspace=0.2)


def stacked_curve_figures():
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, facecolor='white',figsize=(27,6))
    
    x_range, y_range = invariant_curve(invariant2, invariant_liq_display2, invariant_liq2, [s,t])
    invariant = ax1.plot(x_range, y_range, 'k')
    init = ax1.scatter(invariant_liq2[0][1], invariant_liq2[1][1], marker='s', s=70, color='k')
    ax1.set_xlabel(r"$r_{12}$")
    ax1.set_ylabel(r"$r_{21}$")

    x_range, y_range = invariant_curve(invariant3, invariant_liq_display3, invariant_liq3, [x,y])
    invariant = ax2.plot(x_range, y_range, 'k')
    init = ax2.scatter(invariant_liq3[0][1], invariant_liq3[1][1], marker='s', s=70, color='k')
    ax2.set_xlabel(r"$r'_{12}$")
    ax2.set_ylabel(r"$r'_{21}$")

    invariant = stack_pools_parametric(invariant2, invariant3, invariant_liq2, invariant_liq3)
    invariant_liqs = invariant(spot_price(invariant2, invariant_liq2))
    stacked_liq_xr, stacked_liq_yr = invariant_curve_parametric(invariant, invariant_liq_displays)
    invariant = ax3.plot(stacked_liq_xr, stacked_liq_yr, 'k')
    init = ax3.scatter(invariant_liqs[0], invariant_liqs[1], marker='s', s=70, color='k')
    ax3.set_xlabel(r"$r^\Sigma_{12}$")
    ax3.set_ylabel(r"$r^\Sigma_{21}$")
    # plt.text(x=9,y=40, s=r"$\psi_{13} \bigoplus \psi_{32} $", size=25)
    # plt.legend([invariant, init], ['Invariance Curve', 'Initial Liquidity'])
    plt.subplots_adjust(wspace=0.6)

    plt.show()
    fig.savefig('outputs/stacked_curve.png')

def portfolio_decision():
    theta1, theta2, theta3 = symbols("th1 th2 th3")
    p1, p2, p3 = 1, 2, 0.3
    endow1 = 7
    RES = 15
    RES2 = 10
    invariant1 = x * y 
    invariant2 = s * t 

    liq = [[[2,4], [2,8]], [[5,10], [5,20]], [[10,20], [10,40]]]
    cov_list = np.array([
        [[0.2,-0.1],[-0.1,0.2]],
        [[0.2,-0.05],[-0.05,0.2]],
        [[0.2,0],[0,0.2]],
        [[0.2,0.05],[0.05,0.2]],
        [[0.2,0.1],[0.1,0.2]],

    ])
    con_matrices = [] 
    data_range1 = []

    U_f1 = 2*b + 4*c
    triangle = np.transpose(np.array([[0,0],[0,RES2],[RES2,RES2/2]]))

    t2_range = np.linspace(0, endow1/p2, RES)
    t3_range = np.linspace(0, endow1/p3, RES)

    plt.figure(figsize=(40,6))
   
    for i_cov, cov in enumerate(cov_list):
        l = [[1000,2000], [1000,4000]]
        p3 = 0.52
        invariant_liq1 = [(x, l[0][0]), (y,l[0][1])]
        invariant_liq2 = [(s, l[1][0]), (t,l[1][1])]
        t3_range = np.linspace(0, endow1/p3, RES)
        max_con1 = expected_consumption_2T1S(U_f1, [endow1 - p2 * theta2 - p3 * theta3, theta2, theta3], invariant1, invariant2, invariant_liq1, invariant_liq2, cov, [b, c], mc_samples=30)

        con_matrix1 = np.zeros((RES2+1, RES2+1), dtype=np.float32)

        dr1_max = ((0,0), -1)

        for i in range(t2_range.shape[0]):
            for j in range(t3_range.shape[0]):
                bary = np.array([endow1 - t2_range[i] * p2 - t3_range[j] *p3, t2_range[i] * p2, t3_range[j] * p3])/endow1
                if any([x < 0 for x in bary]):
                    continue
                cart = get_cartesian_from_barycentric(bary, triangle)
                f = max_con1.subs([(theta2, t2_range[i]), (theta3, t3_range[j])]).evalf()  
                if f.is_constant():
                    coords = (int(cart[0]), int(cart[1]))
                    con_matrix1[coords] = f 
                    data_range1 += [f]
                    if f > dr1_max[1]:
                        dr1_max = (coords, f, bary)
        
        ax = plt.subplot(1,5,i_cov+1)
        con_matrices += [con_matrix1]
        ax.scatter(dr1_max[0][1], dr1_max[0][0], s=50, marker='x', color='r')

    for i_cov, _ in enumerate(cov_list):
        ax = plt.subplot(1,5,i_cov+1)
        ax.imshow(con_matrices[i_cov], origin='lower', vmin=min(data_range1), vmax=max(data_range1))

    # for q, l in enumerate(liq):
    #     invariant_liq1 = [(x, l[0][0]), (y,l[0][1])]
    #     invariant_liq2 = [(s, l[1][0]), (t,l[1][1])]
    #     max_con1 = expected_consumption_2T1S(U_f1, [endow1 - p2 * theta2 - p3 * theta3, theta2, theta3], invariant1, invariant2, invariant_liq1, invariant_liq2, [[0.3, 0.2], [0.2,0.3]], [b, c], mc_samples=15)
    #     max_con2 = expected_consumption_2T1S(U_f1, [endow1 - p2 * theta2 - p3 * theta3, theta2, theta3], invariant1, invariant2, invariant_liq1, invariant_liq2, [[0.3, 0], [0,0.3]], [b, c], mc_samples=15)
    #     max_con3 = expected_consumption_2T1S(U_f1, [endow1 - p2 * theta2 - p3 * theta3, theta2, theta3], invariant1, invariant2, invariant_liq1, invariant_liq2, [[0.3, -0.2], [-0.2,0.3]], [b, c], mc_samples=15)

    #     con_matrix1 = np.zeros((RES2+1, RES2+1))
    #     con_matrix2 = np.zeros((RES2+1, RES2+1)) 
    #     con_matrix3 = np.zeros((RES2+1, RES2+1)) 

    #     data_range1 = []
    #     dr1_max = ((0,0), -1)
    #     data_range2 = []
    #     dr2_max = ((0,0), -1)
    #     data_range3 = []
    #     dr3_max = ((0,0), -1)
    #     for i in range(t2_range.shape[0]):
    #         for j in range(t3_range.shape[0]):
    #             bary = np.array([endow1 - t2_range[i] * p2 - t3_range[j] *p3, t2_range[i] * p2, t3_range[j] * p3])/endow1
    #             if any([x < 0 for x in bary]):
    #                 continue
    #             cart = get_cartesian_from_barycentric(bary, triangle)
    #             f = max_con1.subs([(theta2, t2_range[i]), (theta3, t3_range[j])]).evalf()  
    #             if f.is_constant():
    #                 coords = (int(cart[0]), int(cart[1]))
    #                 con_matrix1[coords] = f 
    #                 data_range1 += [f]
    #                 if f > dr1_max[1]:
    #                     dr1_max = (coords, f, bary)

    #             f = max_con2.subs([(theta2, t2_range[i]), (theta3, t3_range[j])]).evalf()  
    #             if f.is_constant():
    #                 coords = (int(cart[0]), int(cart[1]))
    #                 con_matrix2[coords] = f 
    #                 data_range2 += [f]
    #                 if f > dr2_max[1]:
    #                     dr2_max = (coords, f, bary)

    #             f = max_con3.subs([(theta2, t2_range[i]), (theta3, t3_range[j])]).evalf()  
    #             if f.is_constant():
    #                 coords = (int(cart[0]), int(cart[1]))
    #                 con_matrix3[coords] = f 
    #                 data_range3 += [f]
    #                 if f > dr3_max[1]:
    #                     dr3_max = (coords, f, bary)
                


    

    #     data_range = np.concatenate((data_range1, data_range2))
    #     ax1 = plt.subplot(3,3,1 + 3*q)
    #     ax2 = plt.subplot(3,3,2 + 3*q)
    #     ax3 = plt.subplot(3,3,3 + 3*q)
    #     # ax1.imshow(con_matrix1, origin='lower', vmin=min(data_range), vmax=max(data_range))
    #     # ax1.scatter(dr1_max[0][1], dr1_max[0][0], s=50, marker='x')
    #     # ax2.imshow(con_matrix2, origin='lower', vmin=min(data_range), vmax=max(data_range))
    #     # ax2.scatter(dr2_max[0][1], dr2_max[0][0], s=50, marker='x')
    #     # ax3.imshow(con_matrix3, origin='lower', vmin=min(data_range), vmax=max(data_range))
    #     # ax3.scatter(dr3_max[0][1], dr3_max[0][0], s=50, marker='x')
    #     ax1.imshow(con_matrix1, origin='lower', vmin=min(con_matrix1[con_matrix1 > 0]))
    #     ax1.scatter(dr1_max[0][1], dr1_max[0][0], s=50, marker='x')
    #     ax2.imshow(con_matrix2, origin='lower', vmin=min(con_matrix2[con_matrix2 > 0]))
    #     ax2.scatter(dr2_max[0][1], dr2_max[0][0], s=50, marker='x')
    #     ax3.imshow(con_matrix3, origin='lower', vmin=min(con_matrix3[con_matrix3 > 0]))
    #     ax3.scatter(dr3_max[0][1], dr3_max[0][0], s=50, marker='x')

    plt.show()

def liquidity_volatility_figures():
    SAMPLES = 3000
    var1 = 0.1
    var2 = 0.1
    cov = np.linspace(-math.sqrt(var1*var2), math.sqrt(var1*var2), 20)
    var_range = np.linspace(0, 0.3, 20)

    t1, t2, w1, w2 = [10, 10, 3, 5]

    a = symbols('a')
    f_inf = lambda w1p, w2p: max(t1 * w1 / w2 * w2p + t2 * w2p, t2 * w2 / w1 * w1p + t1 * w1p)
    df = invariant_to_delta(invariant2, invariant_liq2, a)
    f_lim = lambda w1p, w2p: opt_swap_2(df, s*w2p+t*w1p, [10,10], [a,s,t], relational=False)[1]

    means_cov = np.zeros((3, len(cov)), dtype=np.float32)
    max_error_cov = 0
    means_var = np.zeros((3, len(var_range)), dtype=np.float32)
    max_error_var = 0

    # for i, c in enumerate(cov):
    #         mu1, sigma1 = liq_expected_payoff_mc([[var1, c], [c, var2]], f_inf, [w1, w2], samples=SAMPLES)
    #         means_cov[0,i] = mu1
    #         max_error_cov = max(max_error_cov, sigma1)
    #         mu2, sigma2 = liq_expected_payoff_mc([[var1, c], [c, var2]], f_lim, [w1, w2], samples=SAMPLES)
    #         means_cov[1,i] = mu2
    #         max_error_var = max(max_error_var, sigma2)
    #         means_cov[2,i] = 80*np.exp(0) 

    # print("Max error (covariance figure): ", max_error_cov/math.sqrt(SAMPLES))
    # fig = plt.figure(facecolor='white',figsize=(9,6))

    # corr = cov/(math.sqrt(var1*var2))
    # inf_liq, = plt.plot(corr, means_cov[0,:], linestyle='--', color='k')
    # lim_liq, = plt.plot(corr, means_cov[1,:], linestyle='-.', color='k')
    # no_liq, = plt.plot(corr, means_cov[2,:], linestyle='-', color='k')

    # plt.xlabel("$\\rho$, Correlation")
    # plt.ylabel('C, Max Consumption')
    # plt.xlim(1, -1)

    # plt.legend([inf_liq, lim_liq, no_liq], ['Infinite Liquidity', 'Limited Liquidity', 'No Liquidity'])
    # plt.show()
    # fig.savefig('outputs/liq_correlation.png')

    for i, v in enumerate(var_range):
            mu1, sigma1 = liq_expected_payoff_mc([[v, 0], [0, v]], f_inf, [w1, w2], samples=SAMPLES)
            means_var[0,i] = mu1
            max_error_var = max(max_error_var, sigma1)
            mu2, sigma2 = liq_expected_payoff_mc([[v, 0], [0, v]], f_lim, [w1, w2], samples=SAMPLES)
            means_var[1,i] = mu2

            max_error_var = max(max_error_var, sigma2)
            means_var[2,i] = 80*math.exp(0) 

    print("Max error (variance figure): ", max_error_var/math.sqrt(SAMPLES))
    fig = plt.figure(facecolor='white',figsize=(9,6))

    inf_liq, = plt.plot(var_range, means_var[0,:], linestyle='--', color='k')
    lim_liq, = plt.plot(var_range, means_var[1,:], linestyle='-.', color='k')
    no_liq, = plt.plot(var_range, means_var[2,:], linestyle='-', color='k')

    plt.xlabel("$\sigma^2$, Variance")
    plt.ylabel('C, Max Consumption')
    plt.xlim(0, 0.3)

    plt.legend([inf_liq, lim_liq, no_liq], ['Infinite Liquidity', 'Limited Liquidity', 'No Liquidity'])
    plt.show()
    fig.savefig('outputs/liq_variance.png')


# Cosmetics
mpl.rc('axes', labelsize=18)
mpl.rc('legend', fontsize=18)


# invar_curve1_figure()
# chained_curve_figures()
# stacked_curve_figures()
# optim_swap_figure()
# portfolio_decision()
liquidity_volatility_figures()

'''
INSIGHTS:
why store in tokens when u can always just exchange them in the future?
liq needs to reflect that
'''


KeyboardInterrupt: 

In [ ]:
from sympy import * 

x, y, p = symbols('x y p')
y = 2 * x + 1 
f = x**2 * y 
p_expr = y/x
x_p = solve(p_expr - p, x)[0]

print(integrate(f.subs(x, x_p), p))
# print(integrate((x**2).subs(x, solve(x, p)[0]) , p))

(1 - p)/(p**2 - 4*p + 4)


In [27]:
# Competitive Equilibrium
from scipy.optimize import minimize, Bounds
from sympy import * 
import numpy as np 
import math 

a, b = symbols('a b')
endow1 = [3,5]
endow2 = [2,4]

coeff1 = [2,3]
coeff2 = [2.2,2.8]

util1 = -exp(-coeff1[0] * a) -exp(-coeff1[1] * b)
util2 = -exp(-coeff2[0] * a) -exp(-coeff2[1] * b)

util1




# ---------------------------

m = 2
n = 10 
F = lambda : np.exp(np.random.normal([0,0], [0.1,0.1]))
θ_1, θ_2 = symbols('θ_1 θ_2')
γ_1, γ_2 = symbols('γ_1 γ_2')
Ω = np.random.random((n,m))
Ω /= np.sum(Ω)
b = np.ones(m)
U = [ Matrix([θ_1 * γ_1, θ_2 * γ_2]).dot(F()) for k in range(n)]

# Marshallian demand
ξ = lambda U_θ, p, θ: (minimize(lambda X: -U_θ.subs({
    θ_1: X[0], θ_2: X[1]
}).evalf() , [0,0], constraints=[
    {"type":"ineq", "fun": lambda X: sum(p * θ - p * X)}
], bounds=Bounds([0,0], [np.inf, np.inf]))).x 

# Price movements 
γ_t = np.array([1.0, 1.0])

# Define sufficient KKT conditions (first and second order)
parse_args = lambda x: ()
lagrangian = lambda y, α, p, μ: sum([α[k] * U[k].subs({
    θ_1: y[k][0], θ_2: y[k][1],
    γ_1: γ_t[0], γ_2: γ_t[1],
}) for k in range(n)]) \
    - sum(p * (np.sum(y, axis=1) - b)) \
        + sum(μ * y)

optimum_vector_constraints = [{"type":"ineq", "fun": lambda X: (lambda y, α, p, μ: α[k])(parse_args(X)) } for k in range(n) ] # element-wise coefficients must be >= 0
# complementary slack conditions
slack_conditions1 = [{"type":"ineq", "fun": lambda X: (lambda y, α, p, μ: p[i])(parse_args(X)) } for i in range(2) ] 
slack_conditions2 = [{"type":"eq", "fun": lambda X: (lambda y, α, p, μ: p[i](np.sum(y[:,i] - b[i])))(parse_args(X)) } for i in range(2) ] 

competitive_equilibrium = {"type":"eq", "fun": lambda X: (lambda y, α, p, μ: sum([ξ(U[k].subs({
    γ_1: γ_t[0], γ_2: γ_t[1],
}), p, y[k]) for k in range(n)]) - b)(parse_args(X))} 

solution = minimize(lagrangian, 0, method='SLSQP', constraints = optimum_vector_constraints + slack_conditions1 + slack_conditions2 + [competitive_equilibrium]) 
print(solution)


array([21.25000029,  0.        ])